In [1]:
from dependencies import *
cpidata = pd.read_excel('cpidata.xls',sheet_name='Sheet2')

a = pd.read_stata('table2_1.dta')
b = pd.read_stata('table2_2.dta')
c = pd.read_stata('table3.dta')
d = pd.read_stata('table4.dta')

data = pd.read_stata('table2_1.dta')
data = data.merge(pd.read_stata('table2_2.dta'),on=['month','year','type_fund'])
data = data.merge(pd.read_stata('table3.dta'),on=['month','year','type_fund'])
data = data.merge(pd.read_stata('table4.dta'),on=['month','year','type_fund'])

data['month'] = ['January' if x == 'Janurary' else x for x in data['month']]
data['monthyear'] = [a+str(b) for a,b in zip(data['month'].values,data['year'].values)]
data['date'] = pd.to_datetime(data['monthyear'],infer_datetime_format=True)

data = data.merge(cpidata,on=['date'])
data = data.sort_values(['date','type_fund'])

money_vars = ['new_open_amt', 'new_closed_amt', 'new_interval_amt', 'new_assured_amt', 'exi_open_amt', 'exi_closed_amt', 'exi_interval_amt', 'exi_assured_amt', 'open_red', 'closed_red', 'interval_red', 'assured_re', 'open_aum', 'closed_aum', 'interval_aum', 'assured_aum']



data['new_open_amt'] = data['new_open_amt'].fillna(1)

In [2]:
a1 = [x and y for x,y in zip(data['date'] == 'January2008',data['type_fund'] == 2)]
a2 = [x and y for x,y in zip(data['date'] > 'January2008',data['type_fund'] == 2)]
closednumsum = np.sum(data.loc[a2]['new_closed_no'])
#print('Adding closed num sum', closednumsum)
data.loc[a1,'new_closed_no'] += closednumsum
data.loc[a2,'new_closed_no'] = 0
closedamtsum = np.sum(data.loc[a2]['new_closed_amt'])
#print('Adding closed Amount sum', closedamtsum)
data.loc[a1,'new_closed_amt'] += 4656.147
#data.loc[a1,'new_closed_amt'] += closedamtsum
data.loc[a2,'new_closed_amt'] = 0

In [3]:
for mvar in money_vars:
    data[mvar] = ((data[mvar]/data['cpibase9'])*10)/(46.235)

    
data['net_exi_open_amt'] = data['new_open_amt'].values + data['exi_open_amt'].values - data['open_red'].values
data['net_exi_closed_amt'] = data['new_closed_amt'].values + data['exi_closed_amt'].values - data['closed_red'].values

In [4]:
sensexdata = pd.read_stata('sensex_use.dta')
sensexdata['date'] = pd.to_datetime(sensexdata['month_year'],infer_datetime_format=True)
sensexdata  = sensexdata.set_index('date')
sensexdata['sensex_return1'] = sensexdata['sensex_return'].shift(1).values
sensexdata['sensex_return3'] = ((1+sensexdata['sensex_return'].shift(1).values)*(1+sensexdata['sensex_return'].shift(2).values)*(1+sensexdata['sensex_return'].shift(3).values))-1
#sensexdata['sensex_return1'] = sensexdata['sensex_return'].values
#sensexdata['sensex_return3'] = ((1+sensexdata['sensex_return'].values)*(1+sensexdata['sensex_return'].shift(1).values)*(1+sensexdata['sensex_return'].shift(2).values))-1

sensexdata = sensexdata.reset_index()
sensexdata = sensexdata.drop(columns=['month_year'])

In [5]:
bibordata = pd.read_stata('bibordata.dta')
bibordata['date'] = pd.to_datetime(bibordata['month_year'],infer_datetime_format=True)
bibordata = bibordata.drop(columns=['month_year'])

In [6]:
data = data.merge(sensexdata,on=['date'])
data = data.merge(bibordata,on=['date'],how='left')

In [7]:
maindata = data[['date', 'type_fund', 'new_open_no', 'net_exi_open_amt' ,'new_closed_no', 'net_exi_closed_amt', 'new_interval_no', 'new_interval_amt', 'new_assured_no', 'new_assured_amt','sensex_return','sensex_return1','sensex_return3','bid']]
#maindata = data[['date', 'type_fund', 'new_open_no', 'new_open_amt' ,'new_closed_no', 'new_closed_amt', 'new_interval_no', 'new_interval_amt', 'new_assured_no', 'new_assured_amt','sensex_return','sensex_return1','sensex_return3','bid']]
maindata.columns = ['date', 'type_fund', 'starts1', 'amount1', 'starts2', 'amount2', 'starts3', 'amount3', 'starts4', 'amount4','sensex_return','sensex_return1','sensex_return3','bid']

In [8]:
mainlong = pd.wide_to_long(maindata,stubnames = ['starts','amount'], i=['date','type_fund'],j='structure')
mainlong = mainlong.reset_index()

In [9]:
mainlong = mainlong.sort_values(['date','type_fund'])

In [10]:
mainlong['equity'] = 0
mainlong.loc[mainlong['type_fund'] == 2,'equity']  = 1

In [11]:
mainlong['postdum'] = 0
mainlong.loc[mainlong['date'] > 'January2008','postdum']  = 1

In [12]:
mainlong['regime1'] = 0
mainlong.loc[mainlong['date'] < 'April2006','regime1']  = 1
mainlong['regime2'] = 0
mainlong.loc[ [a and b for a,b in zip(mainlong['date'] >= 'April2006',mainlong['date'] <= 'January2008')],'regime2']  = 1
mainlong['regime3'] = 0
mainlong.loc[mainlong['date'] > 'January2008','regime3']  = 1

In [13]:
mainlong['closed'] = 0
mainlong.loc[mainlong['structure'] == 2,'closed']  = 1

In [14]:
mainlong['closed_regime1'] = mainlong['closed'] * mainlong['regime1']
mainlong['closed_regime2'] = mainlong['closed'] * mainlong['regime2']
mainlong['closed_regime3'] = mainlong['closed'] * mainlong['regime3']

In [15]:
from linearmodels import PanelOLS
import math
temp = mainlong[mainlong['type_fund']==2]
temp = temp[[a or b for a,b in zip(temp['structure']==2,temp['structure']==1)]]
basedate = temp['date'].values[0]
temp['time_trend'] = [math.floor(x.days/30.2) for x in (temp['date'] - basedate)]
temp = temp[['time_trend','structure', 'date','amount','starts' ,'sensex_return','sensex_return1' , 'sensex_return3', 'regime2', 'regime3', 'closed' , 'closed_regime2' , 'closed_regime3']]
temp = temp.set_index(['structure', 'date'])
temp.describe()

,time_trend,amount,starts,sensex_return,sensex_return1,sensex_return3,regime2,regime3,closed,closed_regime2,closed_regime3
count,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000
mean,59.500000,124.554294,1.129167,0.010804,0.010251,0.032069,0.183333,0.166667,0.500000,0.091667,0.083333
std,34.712206,376.514624,1.812319,0.079741,0.080110,0.160064,0.387748,0.373457,0.501045,0.289158,0.276963
min,0.000000,-652.697189,0.000000,-0.247461,-0.247461,-0.401967,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29.750000,-6.108682,0.000000,-0.038970,-0.041630,-0.090966,0.000000,0.000000,0.000000,0.000000,0.000000
50%,59.500000,0.000000,0.000000,0.016898,0.016898,0.003803,0.000000,0.000000,0.500000,0.000000,0.000000
75%,89.250000,126.962381,2.000000,0.066283,0.066283,0.146174,0.000000,0.000000,1.000000,0.000000,0.000000
max,119.000000,2536.345734,12.000000,0.256979,0.256979,0.629468,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
formula1 = 'starts ~ time_trend + sensex_return1+ sensex_return3 + regime2 + regime3 +  closed + closed_regime2 + closed_regime3 + 1 '
formula2 = 'amount ~ time_trend + sensex_return1 + sensex_return3 + regime2 + regime3 +  closed + closed_regime2 + closed_regime3 + 1 '

In [17]:
mod = PanelOLS.from_formula(formula1, temp)
res1 = mod.fit(cov_type='clustered')
print(res1)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 starts   R-squared:                        0.3244
Estimator:                   PanelOLS   R-squared (Between):              1.0000
No. Observations:                 240   R-squared (Within):               0.1897
Date:                Thu, May 09 2019   R-squared (Overall):              0.3244
Time:                        21:03:55   Log-likelihood                   -435.68
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      13.867
Entities:                           2   P-value                           0.0000
Avg Obs:                       120.00   Distribution:                   F(8,231)
Min Obs:                       120.00                                           
Max Obs:                       120.00   F-statistic (robust):             17.571
                            

In [18]:
temprest = temp[[a and b for a,b in zip(temp.reset_index()['date'] >= 'June2004', temp.reset_index()['date'] <= 'November2009')]]
mod = PanelOLS.from_formula(formula1, temprest)
res2 = mod.fit()
print(res2)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 starts   R-squared:                        0.3890
Estimator:                   PanelOLS   R-squared (Between):              1.0000
No. Observations:                 128   R-squared (Within):               0.2651
Date:                Thu, May 09 2019   R-squared (Overall):              0.3890
Time:                        21:03:56   Log-likelihood                   -245.80
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      9.4704
Entities:                           2   P-value                           0.0000
Avg Obs:                       64.000   Distribution:                   F(8,119)
Min Obs:                       64.000                                           
Max Obs:                       64.000   F-statistic (robust):             9.4704
                            

In [19]:
import statsmodels.api as sm
from statsmodels.genmod.families import Poisson

In [20]:
pmodel = sm.GLM.from_formula(formula1, temp, family=Poisson())
res3 = pmodel.fit(cov_type='HC0')
print(res3.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 starts   No. Observations:                  240
Model:                            GLM   Df Residuals:                      231
Model Family:                 Poisson   Df Model:                            8
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -264.13
Date:                Thu, 09 May 2019   Deviance:                       244.04
Time:                        21:03:57   Pearson chi2:                     346.
No. Iterations:                    23   Covariance Type:                   HC0
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.6813      0.366     -1.863      0.062      -1.398       0.035
time_trend         0.0292      0.007    

In [21]:
pmodel = sm.GLM.from_formula(formula1, temprest, family=Poisson())
res4 = pmodel.fit(cov_type='HC0')
print(res4.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 starts   No. Observations:                  128
Model:                            GLM   Df Residuals:                      119
Model Family:                 Poisson   Df Model:                            8
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -166.96
Date:                Thu, 09 May 2019   Deviance:                       134.87
Time:                        21:03:57   Pearson chi2:                     146.
No. Iterations:                    23   Covariance Type:                   HC0
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -2.5314      1.187     -2.132      0.033      -4.858      -0.205
time_trend         0.0574      0.018    

In [22]:
from statsmodels.iolib.summary2 import summary_col
dfoutput = summary_col([res3,res4],stars=True,float_format='%0.2f', model_names=['Model-1','Model-2'], info_dict={'N':lambda x: "{0:d}".format(int(x.nobs))})
print(dfoutput)


                Model-1   Model-2 
----------------------------------
Intercept      -0.68*    -2.53**  
               (0.37)    (1.19)   
time_trend     0.03***   0.06***  
               (0.01)    (0.02)   
sensex_return1 -0.31     0.14     
               (1.29)    (1.51)   
sensex_return3 -0.88     -1.40**  
               (0.58)    (0.67)   
regime2        -1.25***  -1.91*** 
               (0.37)    (0.53)   
regime3        -1.78***  -3.05*** 
               (0.48)    (0.89)   
closed         -4.26***  -4.30*** 
               (0.70)    (0.98)   
closed_regime2 4.37***   4.42***  
               (0.78)    (1.02)   
closed_regime3 -20.07*** -19.66***
               (0.76)    (1.02)   
N              240       128      
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [23]:
mod = PanelOLS.from_formula(formula2, temp)
res5 = mod.fit()
print(res5)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 amount   R-squared:                        0.1722
Estimator:                   PanelOLS   R-squared (Between):              1.0000
No. Observations:                 240   R-squared (Within):               0.1348
Date:                Thu, May 09 2019   R-squared (Overall):              0.1722
Time:                        21:03:58   Log-likelihood                   -1740.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      6.0065
Entities:                           2   P-value                           0.0000
Avg Obs:                       120.00   Distribution:                   F(8,231)
Min Obs:                       120.00                                           
Max Obs:                       120.00   F-statistic (robust):             6.0065
                            

In [24]:
mod = PanelOLS.from_formula(formula2, temprest)
res6 = mod.fit()
print(res6)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 amount   R-squared:                        0.2102
Estimator:                   PanelOLS   R-squared (Between):              1.0000
No. Observations:                 128   R-squared (Within):               0.1781
Date:                Thu, May 09 2019   R-squared (Overall):              0.2102
Time:                        21:03:59   Log-likelihood                   -958.10
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3.9580
Entities:                           2   P-value                           0.0003
Avg Obs:                       64.000   Distribution:                   F(8,119)
Min Obs:                       64.000                                           
Max Obs:                       64.000   F-statistic (robust):             3.9580
                            